# Policy Function Iteration, Convergence Criterion, Solvers

Topics:
1. Policy Function Iteration
2. Root-finding with bisection
3. Hill-climbing algorithm

## Policy Function Iteration

In our dynamic programming problem one of the most expensive steps was finding the optimal consumption choice. But, once we find it, we only use the solution once,
$$
V_{n+1}(a) = u(c_n(a)) + \beta V_n((1+r)a + y - c_n(a)).
$$
Then in the next round we compute another policy.

Policy iteration is an improvement-step that repeatedly applies the solution $c_n$ in the Bellman equation before we step into another value function iteration. Denote $V_{n,m}$ the value function after $n$ value function iterations and $m$ policy function iterations. Then,
$$
V_{n,m+1}(a) = u(c_{n}(a)) + \beta V_{n,m}((1+r)a + y - c_n(a)).
$$

This can significantly speed up finding the solution. But there is a cost in that global convergence is no longer guaranteed.

In [ ]:
# policy function iteration in consumption problem

Another form of policy function iteration iterates directly on the FOC. We know the Euler equation for our model is
\begin{align*}
u'(c(a)) = \beta(1+r)u'(c((1+r)a + y - c(a)))
\end{align*}

We can form a guess $c(a)$ and iterate on the FOC. However, unlike iterating on the value function we generally cannot guarantee convergence. (Why?)

In [ ]:
# iterating on the FOC

## Solvers

Rather than iterating on the FOC, we can also view it as a root-finding problem. A root-finding problem can be cast as finding the $x$ such that $f(x)=0$. This means we can use other mathematical tools to find a solution that might be more effective since convergence through iteration is not guaranteed.

The benchmark root-finding method is called bisection. 